# RAG Evaluation

In this example, we will evaluate our RAG by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of our system.

## Setups

In [ ]:
!pip install -qU torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option('display.max_colwidth', None)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Evaluate RAG performance

For our evaluation pipeline, we will need
1. an evaluation dataset with question - answer couples (QA couples)
2. an evaluator to compute the accuracy of our system on the above evaluation dataset.

We can apply LLMs to help us solve the above two goals:
1. The evaluation dataset will be synthetically generated by an LLM, and questions will be filtered out by other LLMs.
2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) will then perform the evaluation on this synthetic dataset.

### Load our knowledge base

In [ ]:
ds = datasets.load_dataset('m-ric/huggingface_doc', split='train')

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

## Build a synthetic dataset for evaluation

We first build a synthetic dataset of questions and associated contexts. This method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents. Then we set up other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

### Prepare source documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(
        page_content=doc['text'],
        metadata={'source': doc['source']}
    )
    for doc in tqdm(ds)
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=['\n\n', '\n', '.', ' ', '']
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/2647 [00:00<?, ?it/s]

### Setup agents for question generation

We use [`Mixtral-8x7B-Instruct-v0.1`](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) for QA couple generation in this example.

In [ ]:
from huggingface_hub import InferenceClient

repo_id = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
llm_client = InferenceClient(
    model=repo_id,
    timeout=120
)

def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.text_generation(
        prompt,
        max_new_tokens=1000
    )

    return response.strip()


call_llm(llm_client, "This is a test context")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::
"""

Now we can generate our QA couples. In this example, we will generate 10 QA couples and load the rest from the Hub.

For our specific knowledge base, given that we want to get at least ~100 test samples, and accounting for the fact that we will filter out around half of these with our critique agents later on, we should generate much more, in aout >200 samples.

In [ ]:
import random

# limit this for cost and time consideraton
N_GENERATIONS = 10
print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(
        llm_client,
        QA_generation_prompt.format(context=sampled_context.page_content)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split('Answer: ')[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"

        outputs.append({
            'context': sampled_context.page_content,
            'question': question,
            'answer': answer,
            'source_doc': sampled_context.metadata['source']
        })
    except:
        continue

Generating 10 QA couples...


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
pd.DataFrame(outputs).head(3)

,context,question,answer,source_doc
0,"<figure class=""image text-center"">\n <img src=""https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/124_ml-for-games/apiwizard.png"">\n</figure> \n\nNow that the API is set up, you can make calls from your scripts to the API. Let's look at an example of performing a Sentence Similarity task:\n\n```\nusing HuggingFace.API;\n\n/* other code */\n\n// Make a call to the API\nvoid Query() {\n string inputText = ""I'm on my way to the forest."";\n string[] candidates = {\n ""The player is going to the city"",\n ""The player is going to the wilderness"",\n ""The player is wandering aimlessly""\n };\n HuggingFaceAPI.SentenceSimilarity(inputText, OnSuccess, OnError, candidates);\n}\n\n// If successful, handle the result\nvoid OnSuccess(float[] result) {\n foreach(float value in result) {\n Debug.Log(value);\n }\n}\n\n// Otherwise, handle the error\nvoid OnError(string error) {\n Debug.LogError(error);\n}\n\n/* other code */\n```\n\n## Supported Tasks and Custom Models\n\nThe Hugging Face Unity API also currently supports the following tasks:\n\n- [Conversation](https://huggingface.co/tasks/conversational)\n- [Text Generation](https://huggingface.co/tasks/text-generation)\n- [Text to Image](https://huggingface.co/tasks/text-to-image)\n- [Text Classification](https://huggingface.co/tasks/text-classification)\n- [Question Answering](https://huggingface.co/tasks/question-answering)\n- [Translation](https://huggingface.co/tasks/translation)\n- [Summarization](https://huggingface.co/tasks/summarization)\n- [Speech Recognition](https://huggingface.co/tasks/automatic-speech-recognition)\n\nUse the corresponding methods provided by the `HuggingFaceAPI` class to perform these tasks.\n\nTo use your own custom model hosted on Hugging Face, change the model endpoint in the API Wizard.\n\n## Usage Tips",What tasks does the Hugging Face Unity API support?\n,"The Hugging Face Unity API supports the following tasks: Conversation, Text Generation, Text to Image, Text Classification, Question Answering, Translation, Summarization, and Speech Recognition.",huggingface/blog/blob/main/unity-api.md
1,"```py\noutputs = model(**batch)\nprint(outputs.loss, outputs.logits.shape)\n```\n\n```python out\ntensor(0.5441, grad_fn=<NllLossBackward>) torch.Size([8, 2])\n```\n\nAll 🤗 Transformers models will return the loss when `labels` are provided, and we also get the logits (two for each input in our batch, so a tensor of size 8 x 2).\n\nWe're almost ready to write our training loop! We're just missing two things: an optimizer and a learning rate scheduler. Since we are trying to replicate what the `Trainer` was doing by hand, we will use the same defaults. The optimizer used by the `Trainer` is `AdamW`, which is the same as Adam, but with a twist for weight decay regularization (see [""Decoupled Weight Decay Regularization""](https://arxiv.org/abs/1711.05101) by Ilya Loshchilov and Frank Hutter):\n\n```py\nfrom transformers import AdamW\n\noptimizer = AdamW(model.parameters(), lr=5e-5)\n```\n\nFinally, the learning rate scheduler used by default is just a linear decay from the maximum value (5e-5) to 0. To properly define it, we need to know the number of training steps we will take, which is the number of epochs we want to run multiplied by the number of training batches (which is the length of our training dataloader). The `Trainer` uses three epochs by default, so we will follow that:\n\n```py\nfrom transformers import get_scheduler\n\nnum_epochs = 3\nnum_training_steps = num_epochs * len(train_dataloader)\nlr_scheduler = get_scheduler(\n ""linear"",\n optimizer=optimizer,\n num_warmup_steps=0,\n num_training_steps=num_training_steps,\n)\nprint(num_training_steps)\n```\n\n```python out\n1377\n```\n\n### The training loop[[the-training-loop]]\n\nOne last thing: we will want to use the GPU if we have access to one (on a CPU, training might take several hours instead of a couple of minutes). To 

### Setup critique agents

After the previous agent generated questions, we should do a quality check before validating these questions, so we build a critique agent that will rate each question on several crteria according to the paper [*ReST meets ReAct: Self-Improvement for Multi-Step Reasoning LLM Agent*](https://huggingface.co/papers/2312.10003):
* **Groundedness** - Can the question be answered from the given context?
* **Relevance** - Is this question relevant to users? For example, `"What is the date when transformers 4.29.1 was released?"` is not relevant for ML practitioners.

Another failure case is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, for example, `"What is the name of the function used in this guide?"`. Hence, we also build a critique agent for this criteria:
* **Standalone** - Is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be `"What is the function used in this article?"` for a question generated from a specific article.


We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we will eliminate the question from our eval dataset.

When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output raionale give the model more tokens to think and elaborate an answer before summarizing it into a single score token.

Now that we have defined everything, we can build and run these critique agents.

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer:::
"""

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer:::
"""

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer:::
"""

In [ ]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        'groundedness': call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(
                context=output['context'],
                question=output['question']
            )
        ),
        'relevance': call_llm(
            llm_client,
            question_relevance_critique_prompt.format(
                question=output['question']
            )
        ),
        'standalone': call_llm(
            llm_client,
            question_standalone_critique_prompt.format(
                question=output['question']
            )
        )
    }

    try:
        for criterion, evaluation in evaluations.items():
            # key - criterion, value - evaluation
            score, eval_res = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1]
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval_res
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Now we need to filter out bad questions based on our critique agent scores:

In [ ]:
import pandas as pd

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            'question',
            'answer',
            'groundedness_score',
            'relevance_score',
            'standalone_score'
        ]
    ]
)

generated_questions = generated_questions.loc[
    (generated_questions['groundedness_score'] >= 4)
    & (generated_questions['relevance_score'] >= 4)
    & (generated_questions['standalone_score'] >= 4)
]
print('='*50)
print('Final evaluation dataset:')
display(
    generated_questions[
        [
            'question',
            'answer',
            'groundedness_score',
            'relevance_score',
            'standalone_score'
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions,
    split='train',
    preserve_index=False
)

Now our synthetic evaluation dataset is complete. We can evaluate different RAG systems on this evaluation dataset.

We have generated only a few QA couples in this example to reduce time and cost. To move forward, we will load a pre-generated dataset:

In [ ]:
eval_dataset = datasets.load_dataset('m-ric/huggingface_doc_eval', split='train')

In [ ]:
eval_dataset

## Build our RAG system

### Preprocess documents to build our vector database

We will split the documents from our knowledge base into smaller chunks. These will be the snippets that we picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.

The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.

There are many options for text splitting:
* split every `n` words/characters, but this has the risk of cutting in half paragraphs or even sentences
* split after `n` words/characters, but only on sentence boundaries
* **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).

We can visualize how different splitting options affect the chunks we get in the [chunk visualizer](https://huggingface.co/spaces/m-ric/chunk_visualizer) space.

To measure chunk length in out text splitter, our length function will not be the count of characters, but the **count of tokens in the tokenized text**: for subsequent embedder that processes token, measuring length in tokens is more relevant and empirically performs better.

In [ ]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(
        page_content=doc['text'],
        metadata={'source': doc['source']}
    )
    for doc in tqdm(ds)
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

def split_documents(
        chunk_size: int,
        knowledge_base: List[LangchainDocument],
        tokenizer_name: str
) -> List[LangchainDocument]:
    """Split documents into chunks of size `chunk_size` characters
    and return a list of documents
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=['\n\n', '\n', '.', ' ', '']
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

### Retriever - embeddings

The **retriever acts like an internal search engine**: given the user query, it returns the most relevant documents from our knowledge base.

In this example, we will use Langchain vector databases since it offers a convenient FAISS index and allows us to keep document metadata throughout the processing.

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

def load_embeddings(
        langchain_docs: List[LangchainDocument],
        chunk_size: int,
        embedding_model_name: Optional[str] = 'thenlper/gte-small'
) -> FAISS:
    """Create a FAISS index from the given embedding model and documents.
    Loads the index directly if it already exists.

    Parameters
    ----------
    langchain_docs: List[LangchainDocument]
        Documents to be indexed.
    chunk_size: int
        Size of the chunks to split the documents into
    embedding_model_name: name of the embedding model to use

    Returns
    -------
    FAISS index
    """
    # load embedding model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True} # set True to compute cosine similarity
    )

    # check if embeddings already exist on disk
    index_name = f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    index_folder_path = f"./data/indexes/{index_name}"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE
        )
    else:
        print("Index not found, generating it...")
        docs_process = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name
        )

        knowledge_index = FAISS.from_documents(
            docs_process,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)

        return knowledge_index

### Reader - LLM

The Reader LLM reads the retrieved documents to formulate its answer.

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [ ]:
from langchain_community.llms import HuggingFaceHub

repo_id = 'HuggingFaceH4/zephyr-7b-beta'
READER_MODEL_NAME = 'zephyr-7b-beta'
#HP_API_TOEKN = ""

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task='text-generation',
    #huggingfacehub_api_token=HP_API_TOEKN,
    model_kwargs={
        'max_new_tokens': 512,
        'top_k': 30,
        'temperature': 0.1,
        'repetition_penalty': 1.03
    }
)

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM

def answer_with_rag(
        question: str,
        llm: LLM,
        knowledge_index: VectorStore,
        reranker: Optional[RAGPretrainedModel] = None,
        num_retrieved_docs: int = 30,
        num_docs_final: int = 7
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question,
        k=num_retrieved_docs
    )
    # only keep the text
    relevant_docs = [doc.page_content for doc in relevant_docs]

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(
            question,
            relevant_docs,
            k=num_docs_final
        )
        relevant_docs = [doc['content'] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([
        f"Document {str(i)}:::\n" + doc
        for i, doc in enumerate(relevant_docs)
    ])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Generate an answer
    answer = llm(final_prompt)

    return answer, relevant_docs

## Benchmarking the RAG system

The RAG system and the evaluation datasets are ready. The last step is to judge the output of the RAG system on this evaluation dataset. We need to set up a judge agent.

Among different RAG evaluation metrics, we choose to focus only on faithfulness since it the best end-to-end metric of our system's performance.

Here we use GPT4 as a judge for its empirically good performance, but we could try other models such as `kaist-ai/prometheus-13b-v1.0` or `BAAI/JudgeLM-33B-v1.0`.

In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in [**Prometheus' prompt template**](https://huggingface.co/prometheus-eval/prometheus-13b-v1.0): this helps the model ground its metric precisely. If instead we give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples.

Don't forget that *Prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement*.

In [ ]:
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
        eval_dataset: datasets.Dataset,
        llm: BaseChatModel,
        knowledge_index: VectorStore,
        output_file: str,
        reranker: Optional[RAGPretrainedModel] = None,
        verbose: Optional[bool] = True,
        test_settings: Optional[str] = None,
):
    """Run RAG tests on the given dataset and saves the results to the given output file"""
    try:
        # load previous generation if they exist
        with open(output_file, 'r') as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example['question']
        if question in [output['question'] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question,
            llm,
            knowledge_index,
            reranker=reranker
        )


        result = {
            'question': question,
            'true_answer': example['answer'],
            'source_doc': example['source_doc'],
            'generated_answer':answer,
            'retrieved_docs': [doc for doc in relevant_docs]
        }
        if test_settings:
            result['test_settings'] = test_settings
        outputs.append(result)

        with open(output_file, 'w') as f:
            json.dump(outputs, f)

In [2]:
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import SystemMessage

EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You are a fair evaluator language model.'),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT)
    ]
)
evaluation_prompt_template

ChatPromptTemplate(input_variables=['instruction', 'reference_answer', 'response'], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are a fair evaluator language model.', additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['instruction', 'reference_answer', 'response'], input_types={}, partial_variables={}, template='###Task Description:\nAn instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.\n1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.\n2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.\n3. The output format should look as follows: "Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and

In [ ]:
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY = ''

eval_chat_model = ChatOpenAI(
    model='gpt-4o',
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)
evaluator_name = 'GPT4'


def evaluate_answers(
        answer_path: str,
        eval_chat_model: ChatOpenAI,
        evaluator_name: str,
        evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluate generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):
        # load previous generations if they exist
        answers = json.load(open(answer_path, 'r'))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment['question'],
            response=experiment['generated_answer'],
            reference_answer=experiment['true_answer']
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [item.strip() for item in eval_result.content.split("[RESULT]")]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, 'w') as f:
            json.dump(answers, f)

In [ ]:
if not os.path.exists('./output'):
    os.mkdir('./output')

for chunk_size in [128, 256]: # add other chunk size (in tokens) as needed
    for embedding_model_name in ['thenlper/gte-small']: # add other embedding models as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embedding:{embedding_model_name.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f'./output/rag_{settings_name}.json'

            print(f"Running evaluation for {settings_name}")

            print('Loading knowledge base embeddings...')
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embedding_model_name
            )

            print('Running RAG...')
            reranker = RAGPretrainedModel.from_pretrained('colbert-ir/colbertv2.0') if rerank else None
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name
            )

            print('Running evaluation...')
            evaluate_answers(
                answer_path=output_file_name,
                eval_chat_model=eval_chat_model,
                evaluator_name=evaluator_name,
                evaluation_prompt_template=evaluation_prompt_template
            )

### Inspect results

In [ ]:
import glob

outputs = []
for file in glob.glob('./output/*.json'):
    output = pd.DataFrame(json.load(open(file, 'r')))
    output['settings'] = file
    outputs.append(output)

result = pd.concat(outputs)

In [ ]:
result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(lambda x: int(x) if isinstance(x, str) else 1)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

In [ ]:
average_scores = result.groupby("settings")["eval_score_GPT4"].mean()
average_scores.sort_values()

### Example results

This is just the result for this example. We should try several diferent method combinations when tuning our RAG systems for other cases.

In [ ]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

In [ ]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()